# IMDB MOVIE CHALLENGE

In [1]:
from selenium import webdriver
import pandas as pd

In [2]:
my_driver = "chromedriver.exe"
op = webdriver.ChromeOptions()
op.add_argument('headless')

main_url = 'https://www.imdb.com/search/title/?genres=action&groups=top_1000&sort=user_rating,desc&view=simple'

#collect in dictionary
content_dict = {'title':[],'rating':[],'total_votes':[],'release_date':[],'duration':[],'description':[],'genres':[],'casts':[],'director':[],'filming_dates':[]}

with open('movies.csv','w') as file:
    file.write(','.join(content_dict.keys()))
    file.write('\n')

k=0
page_number=0
save_freq = 20

#move through the pages
while True:
    print("="*20+"page("+str(page_number)+")"+"="*20)
    main_driver = webdriver.Chrome(my_driver,options=op)
    main_driver.implicitly_wait(1)
    main_driver.get(main_url)
    content =  main_driver.find_elements_by_class_name('lister-item')

    #for each movie do
    for tem_i in range(len(content)):

        #back to base url
        main_driver = webdriver.Chrome(my_driver,options=op)
        main_driver.implicitly_wait(1)
        main_driver.get(main_url)

        #title
        title_year =  content[tem_i].find_element_by_class_name('col-title')
        title = title_year.find_element_by_tag_name('a')
        title = title.text
        content_dict['title'].append( title )

        #rating
        rating =  content[tem_i].find_element_by_class_name('col-imdb-rating')
        rating = rating.find_element_by_tag_name('strong')
        rating = rating.get_attribute("title")
        content_dict['rating'].append( rating.split()[0] )
        content_dict['total_votes'].append( rating.split()[3] )

        #Follow the link of movie i (redirection to movie i page)
        image =  content[tem_i].find_element_by_class_name('lister-item-image')
        image_a = image.find_element_by_tag_name('a')

        #update movie_i_driver url
        movie_i_driver_url = image_a.get_attribute("href")
        main_driver = webdriver.Chrome(my_driver,options=op)
        main_driver.implicitly_wait(1)
        main_driver.get(movie_i_driver_url)

        #general_info
        general_info = main_driver.find_element_by_xpath('.//div[@class="TitleMainBelowTheFoldGroup__TitleMainPrimaryGroup-sc-1vpywau-1 btXiqv ipc-page-grid__item ipc-page-grid__item--span-2"]')


        #release date
        release_date = general_info.find_element_by_xpath('.//section[@data-testid="Details"]')
        release_date = release_date.find_element_by_xpath('.//div[@data-testid="title-details-section"]')
        release_date = release_date.find_element_by_xpath('.//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
        content_dict['release_date'].append( release_date.text.split('(')[0] )

        #duration
        duration = general_info.find_element_by_xpath('.//section[@data-testid="TechSpecs"]')
        duration = duration.find_element_by_xpath('.//div[@data-testid="title-techspecs-section"]')
        duration = duration.find_element_by_xpath('.//li[@data-testid="title-techspec_runtime"]')
        duration = duration.find_element_by_xpath('.//li[@class="ipc-inline-list__item"]')
        content_dict['duration'].append( duration.text )

        #description
        description = general_info.find_element_by_xpath('.//section[@data-testid="Storyline"]')
        description = description.find_element_by_xpath('.//div[@class="Storyline__StorylineWrapper-sc-1b58ttw-0 iywpty"]')
        description = description.find_element_by_xpath('.//div[@data-testid="storyline-plot-summary"]')
        description = description.find_element_by_xpath('.//div[@class="ipc-html-content ipc-html-content--base"]')
        content_dict['description'].append( description.text )

        #genre
        genre = general_info.find_element_by_xpath('.//section[@data-testid="Storyline"]')
        genre = genre.find_element_by_xpath('.//div[@class="Storyline__StorylineWrapper-sc-1b58ttw-0 iywpty"]')
        genre = genre.find_element_by_xpath('.//li[@data-testid="storyline-genres"]')
        genre = genre.find_elements_by_xpath('.//li[@class="ipc-inline-list__item"]')
        genres = [i.text for i in genre]
        content_dict['genres'].append( genres )

        #cast
        cast = general_info.find_element_by_xpath('.//section[@data-testid="title-cast"]')
        cast = cast.find_element_by_xpath('.//div[@class="ipc-shoveler title-cast__grid"]')
        cast = cast.find_element_by_xpath('.//div[@class="ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid"]')
        cast = cast.find_elements_by_xpath('.//div[@data-testid="title-cast-item"]')
        casts = [i.find_element_by_xpath('.//a[@data-testid="title-cast-item__actor"]').text for i in cast]
        content_dict['casts'].append( casts )

        #director
        director = general_info.find_element_by_xpath('.//section[@data-testid="title-cast"]')
        director = director.find_element_by_xpath('.//ul[@class="ipc-metadata-list ipc-metadata-list--dividers-all StyledComponents__CastMetaDataList-y9ygcu-10 cbPPkN ipc-metadata-list--base"]')
        director = director.find_elements_by_tag_name('li')[0]
        director = director.find_element_by_xpath('.//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
        content_dict['director'].append( director.text )

        #filming dates (this information is not present for some films)
        try:
            filming_location = general_info.find_element_by_xpath('.//section[@data-testid="Details"]')
            filming_location = filming_location.find_element_by_xpath('.//div[@data-testid="title-details-section"]')
            filming_location = filming_location.find_element_by_xpath('.//li[@data-testid="title-details-filminglocations"]')
            filming_location = filming_location.find_element_by_xpath('.//a[@class="ipc-metadata-list-item__icon-link"]')

            #update url (redirection to film location url)
            filming_location_url = filming_location.get_attribute("href")
            main_driver = webdriver.Chrome(my_driver,options=op)
            main_driver.implicitly_wait(1)
            main_driver.get(filming_location_url)

            filming_location_info = main_driver.find_element_by_xpath('.//div[@id="wrapper"]')
            filming_location_info = filming_location_info.find_element_by_xpath('.//div[@id="content-2-wide"]')
            filming_location_info = filming_location_info.find_element_by_xpath('.//section[@class="article listo"]')
            filming_location_info = filming_location_info.find_element_by_xpath('.//section[@id="filming_dates"]')
            filming_dates = filming_location_info.find_element_by_xpath('.//li[@class="ipl-zebra-list__item"]')
            content_dict['filming_dates'].append( filming_dates.text )
        except:
            #append None if filming dates is not found
            content_dict['filming_dates'].append( None )
        
        #save data to csv after save_freq iterations
        if k%save_freq == 0 and k != 0:
            
            df = pd.DataFrame(content_dict)
            df.to_csv('movies.csv', mode='a', header=False, index=False)
            print("writen "+str(k)+" items")
            content_dict = {'title':[],'rating':[],'total_votes':[],'release_date':[],'duration':[],'description':[],'genres':[],'casts':[],'director':[],'filming_dates':[]}
        k += 1
    
    #move to next page
    try:
        #back to base url
        main_driver = webdriver.Chrome(my_driver,options=op)
        main_driver.implicitly_wait(1)
        main_driver.get(main_url)
        
        next_page = main_driver.find_element_by_xpath('//body[@id="styleguide-v2"]')
        next_page = main_driver.find_element_by_xpath('//div[@id="wrapper"]')
        next_page = next_page.find_element_by_xpath('.//div[@id="content-2-wide"]')
        next_page = next_page.find_element_by_xpath('.//div[@id="main"]')
        next_page = next_page.find_element_by_xpath('.//div[@class="desc"]')
        next_page = next_page.find_element_by_xpath('.//a[@class="lister-page-next next-page"]')

        #update main_url for next movie page
        main_url = next_page.get_attribute("href")
    except:
        #stop loop when there is no next page
        break
    page_number += 1

#write remaining items to csv
if k%save_freq != 0:
    df = pd.DataFrame(content_dict)
    df.to_csv('movies.csv', mode='a', header=False, index=False)
    print("writen "+str(k)+" items")



print("Done")

====================page(0)====================
writen 20 items
writen 40 items
====================page(1)====================
writen 60 items
writen 80 items
====================page(2)====================
writen 100 items
writen 120 items
writen 140 items
====================page(3)====================
writen 160 items
writen 180 items
writen 189 items
Done


## Load the csv file below

In [5]:
df = pd.read_csv('movies.csv') 
print(df.head())
print("\nData Shape: ",df.shape)

                                               title  rating total_votes  \
0                                    The Dark Knight     9.0   2,421,336   
1      The Lord of the Rings: The Return of the King     8.9   1,711,120   
2                                          Inception     8.8   2,172,390   
3                                          Shershaah     8.8     102,491   
4  The Lord of the Rings: The Fellowship of the Ring     8.8   1,732,391   

         release_date  duration  \
0      July 18, 2008   2h 32min   
1  December 17, 2003   3h 21min   
2      July 16, 2010   2h 28min   
3    August 12, 2021   2h 15min   
4  December 19, 2001   2h 58min   

                                         description  \
0  Set within a year after the events of Batman B...   
1  The final confrontation between the forces of ...   
2  Dom Cobb is a skilled thief, the absolute best...   
3  Based on the life of Captain Vikram Batra who ...   
4  An ancient Ring thought lost for centuries has...